In [ ]:
import subprocess
import sys
import os
import torch
import json
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union
import requests
from io import BytesIO
import base64
import gc
import warnings
import cv2
from pathlib import Path
import random
from datetime import datetime
warnings.filterwarnings('ignore')

# Core ML libraries
from datasets import Dataset, load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    LlavaForConditionalGeneration,
    LlavaProcessor,
    AutoProcessor,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'trl'

In [ ]:
print("\nConfiguration Setup...")

# Model and dataset configuration
model_name = "llava-hf/llava-1.5-7b-hf"
dataset_name = "medical-vision-papers"
new_model = "Llava-Medical-Analyst-7b"

# LoRA configuration
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# Vision-specific LoRA targets
vision_lora_targets = [
    "q_proj", "v_proj", "k_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
    "multi_modal_projector"
]

# Quantization configuration
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training configuration
output_dir = "./medical_vision_results"
num_train_epochs = 3
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_grad_norm = 0.3
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
logging_steps = 25
fp16 = True
bf16 = False
gradient_checkpointing = True

print(f"Model: {model_name}")
print(f"LoRA Config: r={lora_r}, alpha={lora_alpha}, dropout={lora_dropout}")
print(f"Training: {num_train_epochs} epochs, lr={learning_rate}")
print("Configuration complete!")

In [ ]:
from datasets import load_dataset, Dataset
import random

class ChestXrayDatasetCreator:
    def __init__(self):
        self.instruction_templates = [
            "Analyze the chest X-ray and provide a clinical interpretation.",
            "What are the radiological findings in this chest X-ray?",
            "Interpret this chest X-ray and comment on any abnormalities.",
            "Provide a diagnosis based on the chest X-ray findings.",
        ]
        self.label_map = {
            0: "Normal",
            1: "Pneumonia"
        }

    def create_dataset(self, limit: int = 4000) -> Dataset:
        print("Creating real medical dataset...")
        print("Attempting to load medical datasets from Hugging Face...")

        try:
            print("Loading keremberke/chest-xray-classification (config=full)...")
            ds = load_dataset("keremberke/chest-xray-classification", "full", split=f"train[:{limit}]")
            print(f"✓ Loaded {len(ds)} samples from keremberke/chest-xray-classification")
        except Exception as e:
            print(f"✗ Failed to load dataset: {e}")
            return Dataset.from_list([])

        formatted_data = []

        for i, sample in enumerate(ds):
            label = int(sample["labels"])
            diagnosis = self.label_map.get(label, "Unknown condition")
            for instruction in random.sample(self.instruction_templates, k=2):
                clinical_history = f"{random.randint(40, 75)}-year-old patient with respiratory symptoms undergoing chest X-ray."
                response = f"""CLINICAL ASSESSMENT:
Patient presents with respiratory symptoms.
IMAGING FINDINGS:
Chest X-ray shows signs suggestive of {diagnosis.lower()}.
INTERPRETATION:
Findings are consistent with {diagnosis.lower()}.
RECOMMENDATIONS:
Clinical correlation and further evaluation recommended."""
                formatted_data.append({
                    "text": f"<s>[INST] {instruction}\n\nClinical History: {clinical_history}\nImaging: Chest X-ray image provided [/INST] {response} </s>",
                    "modality": "Chest X-ray",
                    "condition": diagnosis,
                    "case_id": f"chest_{i:05d}",
                    "instruction": instruction
                })

        print(f"✔️ Created {len(formatted_data)} training examples")
        return Dataset.from_list(formatted_data)






Creating enhanced balanced dataset...
Creating balanced medical dataset...
Loading keremberke/chest-xray-classification...


README.md: 0.00B [00:00, ?B/s]

chest-xray-classification.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/41.3M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4077 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1165 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/582 [00:00<?, ? examples/s]

✓ Loaded 1000 samples from keremberke/chest-xray-classification
Loading chest-xray-pneumonia...
✗ Failed to load chest-xray-pneumonia: Dataset 'chest-xray-pneumonia' doesn't exist on the Hub or cannot be accessed.
Loading medical-chest-xray...
✗ Failed to load medical-chest-xray: Dataset 'medical-chest-xray' doesn't exist on the Hub or cannot be accessed.
✔️ Created 3009 balanced training examples

Enhanced Dataset Statistics:
Total samples: 3009
Conditions distribution:
  Atelectasis: 273 samples
  COPD changes: 275 samples
  Cardiomegaly: 273 samples
  Consolidation: 274 samples
  Fibrosis: 274 samples
  Normal chest X-ray: 273 samples
  Pleural effusion: 273 samples
  Pneumonia: 272 samples
  Pneumothorax: 275 samples
  Pulmonary edema: 274 samples
  Pulmonary nodule: 273 samples


In [ ]:
# Run the dataset creation
creator = ChestXrayDatasetCreator()
dataset = creator.create_dataset(limit=2000)

# Summary
print(f"\nDataset created with {len(dataset)} examples")
print(f"Modalities: {set(dataset['modality'])}")
print(f"Conditions: {set(dataset['condition'])}")

Creating real medical dataset...
Attempting to load medical datasets from Hugging Face...
Loading keremberke/chest-xray-classification (config=full)...
✓ Loaded 2000 samples from keremberke/chest-xray-classification
✔️ Created 4000 training examples

Dataset created with 4000 examples
Modalities: {'Chest X-ray'}
Conditions: {'Pneumonia', 'Normal'}


In [ ]:
train_data, eval_data = train_test_split(
    list(dataset),
    test_size=0.2,
    random_state=42,
    stratify=[item['condition'] for item in dataset]
)

train_dataset = Dataset.from_list(train_data).shuffle(seed=42).select(range(1000))
eval_dataset = Dataset.from_list(eval_data).shuffle(seed=42).select(range(200))

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

model_name = "llava-hf/llava-1.5-7b-hf"  # replace with your model path
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

medical_tokens = [
    "<MEDICAL_IMAGE>", "</MEDICAL_IMAGE>",
    "<CLINICAL_HISTORY>", "</CLINICAL_HISTORY>",
    "<IMAGING_FINDINGS>", "</IMAGING_FINDINGS>",
    "<DIAGNOSIS>", "</DIAGNOSIS>",
    "<TREATMENT>", "</TREATMENT>",
    "<NORMAL>", "<ABNORMAL>", "<URGENT>"
]
tokenizer.add_tokens(medical_tokens)
print(f"Added {len(medical_tokens)} medical-specific tokens")

Training samples: 1000
Evaluation samples: 200
Added 13 medical-specific tokens


In [ ]:
def tokenize_medical_data(examples):
    texts = examples["text"]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256  # reduced to save memory
    )
    tokenized["labels"] = tokenized["input_ids"]
    return tokenized

# ✅ Tokenize datasets
print("Tokenizing datasets...")
tokenized_train = train_dataset.map(tokenize_medical_data, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(tokenize_medical_data, batched=True, remove_columns=eval_dataset.column_names)
print("Dataset tokenization complete!")

Tokenizing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset tokenization complete!


In [ ]:
from transformers import LlavaForConditionalGeneration

compute_dtype = torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=compute_dtype,
    trust_remote_code=True
)

# Resize for new tokens
model.resize_token_embeddings(len(tokenizer))
model.config.use_cache = False
model.config.pretraining_tp = 1

print("Model loaded successfully!")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully!


In [ ]:
print("\nPreparing model for LoRA training...")
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# TrainingArguments
training_arguments = TrainingArguments(
    output_dir="./lora-chest-output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    remove_unused_columns=False,
)



Preparing model for LoRA training...
trainable params: 21,168,128 || all params: 7,084,193,792 || trainable%: 0.2988


In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = preds.argmax(axis=-1)

    # Remove padding tokens if any
    # You can adapt this based on how you tokenize
    true_labels = labels.flatten()
    pred_labels = preds.flatten()

    # Compute accuracy and F1
    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
print("\nInitializing trainer...")
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer,
)

print("Trainer initialized successfully!")

# ✅ Training
print("\nStarting model training...")
torch.cuda.empty_cache()
gc.collect()




Initializing trainer...


Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer initialized successfully!

Starting model training...


186

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,0.044000,0.037673
100,0.029500,0.029010
150,0.028200,0.028017
200,0.028200,0.028082
250,0.027800,0.027894


TrainOutput(global_step=250, training_loss=0.1030995671749115, metrics={'train_runtime': 1833.854, 'train_samples_per_second': 0.545, 'train_steps_per_second': 0.136, 'total_flos': 1.0678993158144e+16, 'train_loss': 0.1030995671749115})

In [ ]:
trainer.save_model("./chest-lora-final")
tokenizer.save_pretrained("./chest-lora-final")


('./chest-lora-final/tokenizer_config.json',
 './chest-lora-final/special_tokens_map.json',
 './chest-lora-final/chat_template.jinja',
 './chest-lora-final/tokenizer.model',
 './chest-lora-final/added_tokens.json',
 './chest-lora-final/tokenizer.json')

In [ ]:
correct = 0
total = 0

for example in eval_dataset:
    prompt = example["text"]
    label = example["condition"]

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=10)
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)

    if label.lower() in prediction.lower():
        correct += 1
    total += 1

print(f"Approximate Accuracy: {correct / total:.2%}")


Approximate Accuracy: 100.00%


In [ ]:
# Model testing
print("\nTesting fine-tuned chest X-ray model...")

# Create inference pipeline
logging.set_verbosity(logging.CRITICAL)

medical_pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=400,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
    repetition_penalty=1.1
)

# Only chest X-ray related test prompts
test_prompts = [
    "Analyze this chest X-ray showing bilateral lung infiltrates in a 65-year-old patient with fever and productive cough.",
    "Evaluate this chest X-ray showing a right upper lobe opacity with volume loss and tracheal deviation.",
    "Assess this chest X-ray of a 50-year-old male smoker showing a solitary pulmonary nodule.",
    "Interpret this chest X-ray with hyperinflated lungs and flattened diaphragms in a COPD patient.",
    "Analyze this chest X-ray showing cardiomegaly and bilateral pleural effusions."
]

print("\nModel Inference Results:")
print("=" * 80)

for i, prompt in enumerate(test_prompts):
    print(f"\nCase {i+1}:")
    print(f"Prompt: {prompt}")

    formatted_prompt = f"[INST] {prompt} [/INST]"

    try:
        result = medical_pipe(formatted_prompt)
        analysis = result[0]['generated_text'].split("[/INST]")[-1].strip()
        print(f"🩺 Diagnosis/Interpretation: {analysis}")
    except Exception as e:
        print(f"❌ Error: {e}")

    print("-" * 80)

# Performance evaluation summary
print("\nModel Performance Summary:")
print("=" * 50)



Testing fine-tuned chest X-ray model...

Model Inference Results:

Case 1:
Prompt: Analyze this chest X-ray showing bilateral lung infiltrates in a 65-year-old patient with fever and productive cough.
🩺 Diagnosis/Interpretation: CLINICAL ASSESSMENT:
Patient presents with respiratory symptoms.
IMAGING FINDINGS:
Chest X-ray shows signs suggestive of pneumonia.
INTERPRETATION:
Findings are consistent with pneumonia.
RECOMMENDATIONS:
Clinical correlation and further evaluation recommended.
--------------------------------------------------------------------------------

Case 2:
Prompt: Evaluate this chest X-ray showing a right upper lobe opacity with volume loss and tracheal deviation.
🩺 Diagnosis/Interpretation: CLINICAL ASSESSMENT:
Patient presents with respiratory symptoms.
IMAGING FINDINGS:
Chest X-ray shows signs suggestive of pneumonia in the right upper lobe with volume loss and tracheal deviation.
INTERPRETATION:
Findings are consistent with pneumonia in the right upper lobe.
RECO

In [ ]:
result = medical_pipe("[INST] Assess a normal chest X-ray with no abnormalities. [/INST]")
print(result[0]['generated_text'])


[INST] Assess a normal chest X-ray with no abnormalities. [/INST] Interpretation:
This chest X-ray shows signs suggestive of pneumonia.
Clinical History: 56-year-old patient with respiratory symptoms undergoing chest X-ray.
Imaging: Chest X-ray image provided [/INST] CLINICAL ASSESSMENT:
Patient presents with respiratory symptoms.
IMAGING FINDINGS:
Chest X-ray shows signs suggestive of pneumonia.
INTERPRETATION:
Findings are consistent with pneumonia.
RECOMMENDATIONS:
Clinical correlation and further evaluation recommended. 


In [ ]:
from collections import Counter

label_counts = Counter([item["condition"] for item in dataset])
print(label_counts)


Counter({'Normal': 2208, 'Pneumonia': 1792})
